In [43]:
import os
import openai
from openai import OpenAI
import json

In [5]:
client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

In [93]:
system_prompt = """
You are a very experienced Personal Assistant with expertise in drafting out informal emails to from conversations.
Generate an appropriate subject line and body for the text of the conversation provided.  Do not leave any template to be filled.
Generate a complete and thorough email with the context you have."""

In [126]:
generate_email_tool = [
    {
        'type': 'function',
        'function': {
            'name': 'extract_email_details',
            'description': 'Extract/Generate a subject/theme and body for an email from the provided text. Ignore all email addresses in the text.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'subject': {
                        'type': 'string',
                        'items': {
                            'type': 'string',
                        },
                        'description': 'The subject of the email extracted from the text. If multiple emails are found, do not personalise the email. And do not include any extra information. Only details from the text.  Do not leave any template to be filled. Generate a complete and thorough body of the email with the context you have."'
                    },
                    'body': {
                        'type': 'string',
                        'items': {
                            'type': 'string',
                        },
                        'description': 'A fully formatted body of the email with the appropriate greetings and closings.'
                    }
                }
            },
            'required': ['subject', 'body']
        },
        'instruction': system_prompt
    }
]

In [127]:
prompt = "I have a meeting by 6pm today and I want to send an email reminder to my husband to help pick up the kids from school and take them home for me. Draft an email reminder to my husband showing my concern and I would greatly appreciate the help just for today. My husband's email address is samuelbamgbola@gmail.com. And my name is Samuel Bamgbola"

In [128]:
completion = client.chat.completions.create(
    temperature=0.5,
    model="gpt-4o",
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}],
    tools=generate_email_tool,
    tool_choice='required',
)

In [129]:
response = completion.choices[0].message.tool_calls[0].function.arguments

In [133]:
completion.choices[0].message.tool_calls[0].function.arguments

'{"subject": "Reminder to Pick Up Kids from School", "body": "Hi dear,\\n\\nI hope you\'re doing well. I just wanted to send a quick reminder that I have a meeting at 6pm today. Could you please help pick up the kids from school and take them home for me? I would greatly appreciate the help just for today.\\n\\nThank you so much!\\n\\nBest,\\nSamuel"}'

In [130]:
full_response = json.loads(response)

In [131]:
print(full_response["subject"])

Reminder to Pick Up Kids from School


In [132]:
print(full_response["body"])

Hi dear,

I hope you're doing well. I just wanted to send a quick reminder that I have a meeting at 6pm today. Could you please help pick up the kids from school and take them home for me? I would greatly appreciate the help just for today.

Thank you so much!

Best,
Samuel


In [141]:
from PyPDF2 import PdfWriter, PdfReader
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

packet = io.BytesIO()
can = canvas.Canvas(packet, pagesize=letter)
can.drawString(10, 100, "Hello world")
can.save()

#move to the beginning of the StringIO buffer
packet.seek(0)

# create a new PDF with Reportlab
new_pdf = PdfReader(packet)
# read your existing PDF
existing_pdf = PdfReader(open("/Users/mac/Documents/Mailme.AI/letterhead/base.pdf", "rb"))
# packet_2 = io.BytesIO()

# output = PdfReader(packet_2)
# add the "watermark" (which is the new pdf) on the existing page
page = existing_pdf.pages[0]
page.merge_page(new_pdf.pages[0])
output = PdfWriter(open("/Users/mac/Documents/Mailme.AI/letterhead/blank.pdf", "rb"))
# output.add_page(page)
# finally, write "output" to a real file
output_stream = open("destination.pdf", "wb")
output.write(output_stream)
output_stream.close()

In [142]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import io

# Function to add text to PDF
def add_text_to_pdf(input_pdf, output_pdf, text, x, y):
    # Read the existing PDF
    reader = PdfReader(input_pdf)
    writer = PdfWriter()
    
    # Create a new PDF to overlay the text
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    
    # Set the font and size
    can.setFont("Helvetica", 12)
    
    # Draw the text at the specified position (x, y)
    can.setFillColorRGB(0, 0, 0)  # Set text color to black
    can.drawString(x, y, text)
    can.save()
    
    # Move to the beginning of the StringIO buffer
    packet.seek(0)
    
    # Read the overlay PDF
    overlay = PdfReader(packet)
    overlay_page = overlay.pages[0]
    
    # Merge the overlay with the original PDF
    for page in reader.pages:
        page.merge_page(overlay_page)
        writer.add_page(page)
    
    # Save the result to a new PDF
    with open(output_pdf, "wb") as output_file:
        writer.write(output_file)

# Example usage
input_pdf = "/Users/mac/Documents/Mailme.AI/letterhead/base.pdf"  # Path to your letterhead PDF
output_pdf = "destination.pdf"  # Output PDF
text = "This is a sample text"  # Text to add


x, y = 100, 750  # Position of the text (in points)

add_text_to_pdf(input_pdf, output_pdf, text, x, y)


In [21]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import io

# Function to add text to PDF
def add_text_to_pdf(input_pdf, output_pdf, address, subject, body):
    # Read the existing PDF
    reader = PdfReader(input_pdf)
    writer = PdfWriter()
    
    # Create a new PDF to overlay the text
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    
    # Set font and text properties
    can.setFont("Times-Bold", 14)  # Bold font for the subject

    # Calculate the width of the subject string
    subject_width = can.stringWidth(subject, "Times-Bold", 14)
    
    # Center the subject text (x position is half the page width minus half the text width)
    page_width, page_height = letter[0], letter[1]  # Letter page width
    print(page_width, page_height)
    x_subject = (page_width - subject_width) / 2  # Centered position

    # Draw the centered subject
    can.drawString(x_subject, 550, f"{subject}")
    
    # Set font for normal text (address and body)
    can.setFont("Times-Roman", 12)
    
    # Right-align the address
    address_width = can.stringWidth(address, "Times-Roman", 12)
    print(address_width)
    can.drawString(page_width - address_width - 65, 650, address)  # Adjust x position for right-align
    
    # Add the body text
    can.drawString(65, 520, body)
    
    # Save the overlay PDF
    can.save()
    
    # Move to the beginning of the StringIO buffer
    packet.seek(0)
    
    # Read the overlay PDF
    overlay = PdfReader(packet)
    overlay_page = overlay.pages[0]
    
    # Merge the overlay with the original PDF (on the first page only)
    first_page = reader.pages[0]
    first_page.merge_page(overlay_page)
    writer.add_page(first_page)
    
    # If there are multiple pages in the input, add the remaining pages
    for page_num in range(1, len(reader.pages)):
        writer.add_page(reader.pages[page_num])
    
    # Save the result to a new PDF
    with open(output_pdf, "wb") as output_file:
        writer.write(output_file)

# Example usage
input_pdf = "/Users/mac/Documents/Mailme.AI/letterhead/base.pdf"  # Path to your letterhead PDF
output_pdf = "destination.pdf"  # Output PDF

address = "1234 Example St, Example City, Country"  # Right-aligned address
subject = "Important Meeting"  # Subject (Bold, Centered)
body = "This is the body of the email."  # Normal text body

add_text_to_pdf(input_pdf, output_pdf, address, subject, body)


612.0 792.0
200.34


In [33]:
import textwrap
body = """
Hi Babe,

I hope you're doing well. I just wanted to send you a quick reminder that I have a meeting at 6pm today. Could you please help me by picking up the kids from school and taking them home? I would greatly appreciate your help just for today.

Thank you so much!

Love,
Samuel
"""

In [35]:
group = body.split("\n")
for line in group:
    print(textwrap.fill(line, width=100))


Hi Babe,

I hope you're doing well. I just wanted to send you a quick reminder that I have a meeting at 6pm
today. Could you please help me by picking up the kids from school and taking them home? I would
greatly appreciate your help just for today.

Thank you so much!

Love,
Samuel



In [36]:
type(textwrap.fill(line, width=100))

str

In [39]:
textwrap.fill(body, width=100)

" Hi Babe,  I hope you're doing well. I just wanted to send you a quick reminder that I have a\nmeeting at 6pm today. Could you please help me by picking up the kids from school and taking them\nhome? I would greatly appreciate your help just for today.  Thank you so much!  Love, Samuel"

In [40]:
textwrap.fill(body, width=100).split("\n")

[" Hi Babe,  I hope you're doing well. I just wanted to send you a quick reminder that I have a",
 'meeting at 6pm today. Could you please help me by picking up the kids from school and taking them',
 'home? I would greatly appreciate your help just for today.  Thank you so much!  Love, Samuel']

In [43]:
group = body.split("\n")
for line in group:
    all_together = textwrap.fill(line, width=100).split("\n")

    for section in all_together:
        print(textwrap.fill(section, width=100))


Hi Babe,

I hope you're doing well. I just wanted to send you a quick reminder that I have a meeting at 6pm
today. Could you please help me by picking up the kids from school and taking them home? I would
greatly appreciate your help just for today.

Thank you so much!

Love,
Samuel

